In [1]:
import pandas as pd
import random

# -----------------------------
# LOAD PLAYER DATA
# -----------------------------
df_players = pd.read_csv("fantasy_players.csv")

# Normalize column names just in case
df_players.columns = [c.strip() for c in df_players.columns]

# -----------------------------
# ROSTER SETUP
# -----------------------------
def empty_roster():
    return {
        "QB": 0,
        "RB": 0,
        "WR": 0,
        "TE": 0,
        "FLEX": 0,   # can take RB/WR/TE
        "DST": 0,
        "K": 0,
        "BENCH": 0
    }

# Max allowed per slot
MAX_SLOTS = {
    "QB": 1,
    "RB": 2,
    "WR": 2,
    "TE": 1,
    "FLEX": 1,
    "DST": 1,
    "K": 1,
    "BENCH": 6
}

# Position eligibility rules
def slot_can_take(slot, player_pos):
    if slot == "FLEX":
        return player_pos in ["RB", "WR", "TE"]
    return slot == player_pos

# -----------------------------
# PLAYER RECOMMENDER
# -----------------------------
def recommend_player(df, roster, drafted, current_pick, adp_window=12):
    df_available = df[~df['Name'].isin(drafted) & (df['InjuryStatus'] != "Out")]

    # Filter by ADP window
    df_window = df_available[
        (df_available["ADP"] >= max(1, current_pick - adp_window)) &
        (df_available["ADP"] <= current_pick + adp_window)
    ]

    # Pick the best scoring player inside the window if possible
    if not df_window.empty:
        best = df_window.loc[df_window["Score"].idxmax()]
    else:
        # fallback to best available
        best = df_available.loc[df_available["Score"].idxmax()]

    # Decide roster slot to place the player
    pos = best["Position"]
    chosen_slot = None
    for slot in MAX_SLOTS:
        if slot_can_take(slot, pos) and roster[slot] < MAX_SLOTS[slot]:
            chosen_slot = slot
            break

    # If no slot, shove into bench
    if chosen_slot is None:
        chosen_slot = "BENCH"

    # ------------ FLEX DISPLAY FIX ------------
    if chosen_slot == "FLEX":
        display_pos = f"{pos}/FLEX"
    else:
        display_pos = pos
    # ------------------------------------------

    explanation = (
        f"{best['Name']} ({best['Team']} - {display_pos}) - "
        f"Score={best['Score']:.1f}, ADP={best['ADP']}, Injury={best['InjuryStatus']}"
    )

    return best, chosen_slot, explanation

# -----------------------------
# DRAFT SIMULATION
# -----------------------------
teams = int(input("Enter number of teams (e.g., 8): "))
snake = input("Snake draft? (y/n): ").strip().lower() == "y"
ppr = input("PPR scoring? (y/n): ").strip().lower() == "y"
user_slot = int(input(f"Enter your draft slot (1-{teams}): "))

rosters = [empty_roster() for _ in range(teams)]
drafted_players = []

TOTAL_ROUNDS = 15  # extended roster for realism

for rnd in range(TOTAL_ROUNDS):
    print(f"\n=== Round {rnd+1} ===")
    order = list(range(teams))

    if snake and rnd % 2 == 1:
        order.reverse()

    for pick in order:
        current_pick = rnd * teams + (pick + 1)

        if pick == (user_slot - 1):
            # YOUR PICK
            player, slot, explanation = recommend_player(
                df_players, rosters[pick], drafted_players, current_pick
            )
            rosters[pick][slot] += 1
            drafted_players.append(player["Name"])

            print(f"Round {rnd+1}, YOUR TEAM: {explanation}")

        else:
            # AUTODRAFT OTHER TEAMS
            player, slot, _ = recommend_player(
                df_players, rosters[pick], drafted_players, current_pick
            )
            rosters[pick][slot] += 1
            drafted_players.append(player["Name"])

            print(f"Round {rnd+1}, Team {pick+1}: {player['Name']} ({player['Team']} - {player['Position']})")

Enter number of teams (e.g., 8):  8
Snake draft? (y/n):  Y
PPR scoring? (y/n):  Y
Enter your draft slot (1-8):  4



=== Round 1 ===
Round 1, Team 1: Justin Henry 458 (MIN - FLEX)
Round 1, Team 2: Patrick Adams 123 (CLE - WR)
Round 1, Team 3: Derrick Jefferson 482 (DET - FLEX)
Round 1, YOUR TEAM: Stefon Kelce 454 (NYJ - FLEX) - Score=192.1, ADP=16, Injury=Healthy
Round 1, Team 5: Stefon Kelce 331 (GB - TE)
Round 1, Team 6: Davante Diggs 426 (HOU - DEF)
Round 1, Team 7: Davante Mahomes 64 (NYJ - WR)
Round 1, Team 8: Travis Godwin 37 (CAR - WR)

=== Round 2 ===
Round 2, Team 8: Aaron Mahomes 176 (BAL - RB)
Round 2, Team 7: Davante Henry 3 (PIT - WR)
Round 2, Team 6: Justin Jefferson 275 (CHI - QB)
Round 2, Team 5: Lamar Jefferson 197 (DET - RB)
Round 2, YOUR TEAM: Tyreek Godwin 57 (ARI - WR) - Score=121.0, ADP=6, Injury=Healthy
Round 2, Team 3: Travis Rodgers 294 (SEA - QB)
Round 2, Team 2: Travis Rodgers 1 (SEA - WR)
Round 2, Team 1: Patrick Rodgers 153 (SEA - RB)

=== Round 3 ===
Round 3, Team 1: Derrick Kelce 323 (SEA - TE)
Round 3, Team 2: Chris Rodgers 163 (NYJ - RB)
Round 3, Team 3: Tyreek Hill 